In [1]:
# import all relevant libraries
import pyodbc # to access Ms Access database

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# connect to Ms Access database

db_path = r"C:\Users\GILBERT FG\Desktop\Readings\PSP_database.accdb" # path to Ms access database

conn = pyodbc.connect(
    r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};"
    rf"DBQ={db_path};"
)

cursor = conn.cursor()

# list all tables
for table in cursor.tables(tableType="TABLE"):
    print(table.table_name)


Block register
Plot monitoring history
Plot-block relation
PSPs
Slope correction factors
Thinning history
TreeData


In [3]:
# Queries
# Select all from Tree Data and filter the teak trees from Tain II forest reserve
tree_data_query = r"SELECT * FROM TreeData WHERE AreaType = 'Teak' AND Plantations = 'Tain II'"

# read the dataset into dataframe
df = pd.read_sql(tree_data_query, conn)

# preview the dataset
df.head()

C:\Users\GILBERT FG\AppData\Local\Temp\ipykernel_14964\4228675863.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(tree_data_query, conn)


,Plantations,AreaType,Monitoring year,Monitoring month,Monitoring day,PLOT,TREE NR,Tree SPECIES,Species scientific name,H (m),DBH (cm),Merchantable height (m),REMARKS,Incorrect DBH,Incorrect H,Incorrect H / DBH,Exclude,Justification for exclusion
0,Tain II,Teak,2018.0,1.0,13,1,71.0,Teak,Tectona grandis,7.25,12.0,None,Fire scars,False,False,None,False,None
1,Tain II,Teak,2018.0,1.0,13,1,76.0,Teak,Tectona grandis,8.25,12.0,None,Fire scars,False,False,None,False,None
2,Tain II,Teak,2018.0,1.0,13,1,68.0,Teak,Tectona grandis,7.00,11.8,None,Fire scars,False,False,None,False,None
3,Tain II,Teak,2018.0,1.0,13,1,53.0,Teak,Tectona grandis,7.25,11.5,None,Fire scars,False,False,None,False,None
4,Tain II,Teak,2018.0,1.0,13,1,58.0,Teak,Tectona grandis,7.00,10.5,None,Fire scars,False,False,None,False,None


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191860 entries, 0 to 191859
Data columns (total 18 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Plantations                  191860 non-null  object 
 1   AreaType                     191860 non-null  object 
 2   Monitoring year              191860 non-null  float64
 3   Monitoring month             191860 non-null  float64
 4   Monitoring day               191860 non-null  int64  
 5   PLOT                         191860 non-null  object 
 6   TREE NR                      191860 non-null  float64
 7   Tree SPECIES                 191860 non-null  object 
 8   Species scientific name      191851 non-null  object 
 9   H (m)                        191804 non-null  float64
 10  DBH (cm)                     191800 non-null  float64
 11  Merchantable height (m)      0 non-null       object 
 12  REMARKS                      70254 non-null   object 
 13 

# DATA CLEANING

### Trim the dataset into relevant features for the analysis

In [5]:
dtable = df[['Plantations', 'AreaType', 'Monitoring year', 'Monitoring month', 'Monitoring day', 'PLOT', 'TREE NR', 'Tree SPECIES', 'H (m)', 'DBH (cm)', 'REMARKS']]
dtable.head()

,Plantations,AreaType,Monitoring year,Monitoring month,Monitoring day,PLOT,TREE NR,Tree SPECIES,H (m),DBH (cm),REMARKS
0,Tain II,Teak,2018.0,1.0,13,1,71.0,Teak,7.25,12.0,Fire scars
1,Tain II,Teak,2018.0,1.0,13,1,76.0,Teak,8.25,12.0,Fire scars
2,Tain II,Teak,2018.0,1.0,13,1,68.0,Teak,7.00,11.8,Fire scars
3,Tain II,Teak,2018.0,1.0,13,1,53.0,Teak,7.25,11.5,Fire scars
4,Tain II,Teak,2018.0,1.0,13,1,58.0,Teak,7.00,10.5,Fire scars


### Delete all records with no height and dbh

In [6]:
dtable.isna().sum()

Plantations              0
AreaType                 0
Monitoring year          0
Monitoring month         0
Monitoring day           0
PLOT                     0
TREE NR                  0
Tree SPECIES             0
H (m)                   56
DBH (cm)                60
REMARKS             121606
dtype: int64

In [7]:
dtable = dtable.dropna(subset=['H (m)', 'DBH (cm)'])

In [8]:
dtable.head(15)

,Plantations,AreaType,Monitoring year,Monitoring month,Monitoring day,PLOT,TREE NR,Tree SPECIES,H (m),DBH (cm),REMARKS
0,Tain II,Teak,2018.0,1.0,13,1,71.0,Teak,7.25,12.0,Fire scars
1,Tain II,Teak,2018.0,1.0,13,1,76.0,Teak,8.25,12.0,Fire scars
2,Tain II,Teak,2018.0,1.0,13,1,68.0,Teak,7.00,11.8,Fire scars
3,Tain II,Teak,2018.0,1.0,13,1,53.0,Teak,7.25,11.5,Fire scars
4,Tain II,Teak,2018.0,1.0,13,1,58.0,Teak,7.00,10.5,Fire scars
5,Tain II,Teak,2018.0,1.0,13,1,56.0,Teak,6.25,10.4,Fire scars
6,Tain II,Teak,2018.0,1.0,13,1,72.0,Teak,6.50,9.9,Fire scars
7,Tain II,Teak,2018.0,1.0,13,1,78.0,Teak,6.00,9.2,Fire scars
8,Tain II,Teak,2018.0,1.0,13,1,19.0,Teak,4.50,8.1,Fire scars
9,Tain II,Teak,2018.0,1.0,13,1,38.0,Teak,4.00,7.7,Fire scars


In [9]:
dtable.isna().sum()

Plantations              0
AreaType                 0
Monitoring year          0
Monitoring month         0
Monitoring day           0
PLOT                     0
TREE NR                  0
Tree SPECIES             0
H (m)                    0
DBH (cm)                 0
REMARKS             121599
dtype: int64

### Remove all rows with fire scars, beetle attack, top broken, dieback, forked, crooked and half dead

In [10]:
dtable['REMARKS'].unique()

array(['Fire scars', None, 'Beetle infestation', 'Top broken',
       'Top Broken', 'Climber on top', 'Beetle', 'Bent', 'Dieback',
       'climber', 'forked', 'bent', '', 'marked', 'Top Broken, Marked',
       'under a tree', 'Climber', 'Forked', 'tall', 'fork', 'short',
       'Abnormal tree, correct measurement', 'Under tree', 'crooked', ' ',
       'Half dead', 'Under a tree', 'TOP BROKEN', 'Marked',
       'Abnormal tree', 'Crooked'], dtype=object)

In [11]:
dtable = dtable[~((dtable['REMARKS'] == 'Fire scars') | (dtable['REMARKS'] == 'Beetle infestation') | (dtable['REMARKS'] == 'Top broken'))]

In [12]:
dtable = dtable[~((dtable['REMARKS'] == 'Top Broken') | (dtable['REMARKS'] == 'Beetle')  | (dtable['REMARKS'] == 'Bent') | (dtable['REMARKS'] == 'Dieback'))]

In [13]:
dtable.info()

<class 'pandas.core.frame.DataFrame'>
Index: 187182 entries, 44 to 191859
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Plantations       187182 non-null  object 
 1   AreaType          187182 non-null  object 
 2   Monitoring year   187182 non-null  float64
 3   Monitoring month  187182 non-null  float64
 4   Monitoring day    187182 non-null  int64  
 5   PLOT              187182 non-null  object 
 6   TREE NR           187182 non-null  float64
 7   Tree SPECIES      187182 non-null  object 
 8   H (m)             187182 non-null  float64
 9   DBH (cm)          187182 non-null  float64
 10  REMARKS           65583 non-null   object 
dtypes: float64(5), int64(1), object(5)
memory usage: 17.1+ MB


In [14]:
dtable = dtable[~((dtable['REMARKS'] == 'TOP BROKEN'))]

In [15]:
dtable = dtable[~((dtable['REMARKS'] == 'forked') | (dtable['REMARKS'] == 'Forked') | (dtable['REMARKS'] == 'fork'))]

In [16]:
dtable.info()

<class 'pandas.core.frame.DataFrame'>
Index: 187164 entries, 44 to 191859
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Plantations       187164 non-null  object 
 1   AreaType          187164 non-null  object 
 2   Monitoring year   187164 non-null  float64
 3   Monitoring month  187164 non-null  float64
 4   Monitoring day    187164 non-null  int64  
 5   PLOT              187164 non-null  object 
 6   TREE NR           187164 non-null  float64
 7   Tree SPECIES      187164 non-null  object 
 8   H (m)             187164 non-null  float64
 9   DBH (cm)          187164 non-null  float64
 10  REMARKS           65565 non-null   object 
dtypes: float64(5), int64(1), object(5)
memory usage: 17.1+ MB


In [17]:
dtable['REMARKS'].unique()

array([None, 'Climber on top', 'climber', 'bent', '', 'marked',
       'Top Broken, Marked', 'under a tree', 'Climber', 'tall', 'short',
       'Abnormal tree, correct measurement', 'Under tree', 'crooked', ' ',
       'Half dead', 'Under a tree', 'Marked', 'Abnormal tree', 'Crooked'],
      dtype=object)

In [18]:
dtable = dtable[~((dtable['REMARKS'] == 'crooked') | (dtable['REMARKS'] == 'Crooked') | (dtable['REMARKS'] == 'bent') | (dtable['REMARKS'] == 'Top Broken') | (dtable['REMARKS'] == 'Half dead'))]


In [19]:
dtable.info()

<class 'pandas.core.frame.DataFrame'>
Index: 187158 entries, 44 to 191859
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Plantations       187158 non-null  object 
 1   AreaType          187158 non-null  object 
 2   Monitoring year   187158 non-null  float64
 3   Monitoring month  187158 non-null  float64
 4   Monitoring day    187158 non-null  int64  
 5   PLOT              187158 non-null  object 
 6   TREE NR           187158 non-null  float64
 7   Tree SPECIES      187158 non-null  object 
 8   H (m)             187158 non-null  float64
 9   DBH (cm)          187158 non-null  float64
 10  REMARKS           65559 non-null   object 
dtypes: float64(5), int64(1), object(5)
memory usage: 17.1+ MB


In [20]:
dtable['REMARKS'].unique()

array([None, 'Climber on top', 'climber', '', 'marked',
       'Top Broken, Marked', 'under a tree', 'Climber', 'tall', 'short',
       'Abnormal tree, correct measurement', 'Under tree', ' ',
       'Under a tree', 'Marked', 'Abnormal tree'], dtype=object)

In [21]:
dtable = dtable[~((dtable['REMARKS'] == 'Top Broken, Marked'))]

In [22]:
dtable['REMARKS'].unique()

array([None, 'Climber on top', 'climber', '', 'marked', 'under a tree',
       'Climber', 'tall', 'short', 'Abnormal tree, correct measurement',
       'Under tree', ' ', 'Under a tree', 'Marked', 'Abnormal tree'],
      dtype=object)

In [23]:
dtable.info()

<class 'pandas.core.frame.DataFrame'>
Index: 187156 entries, 44 to 191859
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Plantations       187156 non-null  object 
 1   AreaType          187156 non-null  object 
 2   Monitoring year   187156 non-null  float64
 3   Monitoring month  187156 non-null  float64
 4   Monitoring day    187156 non-null  int64  
 5   PLOT              187156 non-null  object 
 6   TREE NR           187156 non-null  float64
 7   Tree SPECIES      187156 non-null  object 
 8   H (m)             187156 non-null  float64
 9   DBH (cm)          187156 non-null  float64
 10  REMARKS           65557 non-null   object 
dtypes: float64(5), int64(1), object(5)
memory usage: 17.1+ MB


### Create monitoring date with a date/time type

In [24]:
# convert the data types of monitoring year and month to integer type to remove the decimal from the floating point
dtable['Monitoring year'] = dtable['Monitoring year'].astype(int)
dtable['Monitoring month'] = dtable['Monitoring month'].astype(int)

In [25]:
# convert the monitoring year, month and day to str type to create date from them
dtable['Monitoring year'] = dtable['Monitoring year'].astype(str)
dtable['Monitoring month'] = dtable['Monitoring month'].astype(str)
dtable['Monitoring day'] = dtable['Monitoring day'].astype(str)

dtable.info()

<class 'pandas.core.frame.DataFrame'>
Index: 187156 entries, 44 to 191859
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Plantations       187156 non-null  object 
 1   AreaType          187156 non-null  object 
 2   Monitoring year   187156 non-null  object 
 3   Monitoring month  187156 non-null  object 
 4   Monitoring day    187156 non-null  object 
 5   PLOT              187156 non-null  object 
 6   TREE NR           187156 non-null  float64
 7   Tree SPECIES      187156 non-null  object 
 8   H (m)             187156 non-null  float64
 9   DBH (cm)          187156 non-null  float64
 10  REMARKS           65557 non-null   object 
dtypes: float64(3), object(8)
memory usage: 17.1+ MB


In [26]:
dtable['Measuring date'] = dtable['Monitoring day'] + '/' + dtable['Monitoring month'] + '/' + dtable['Monitoring year']
dtable.head()

,Plantations,AreaType,Monitoring year,Monitoring month,Monitoring day,PLOT,TREE NR,Tree SPECIES,H (m),DBH (cm),REMARKS,Measuring date
44,Tain II,Teak,2019,1,22,1,68.0,Teak,9.50,14.8,None,22/1/2019
45,Tain II,Teak,2019,1,22,1,76.0,Teak,7.50,14.5,None,22/1/2019
46,Tain II,Teak,2019,1,22,1,71.0,Teak,7.25,14.3,None,22/1/2019
47,Tain II,Teak,2019,1,22,1,53.0,Teak,7.50,13.3,None,22/1/2019
48,Tain II,Teak,2019,1,22,1,58.0,Teak,8.00,13.0,None,22/1/2019


In [27]:
# convert the measuring date data type from str to datetime
dtable['Measuring date'] = pd.to_datetime(dtable['Measuring date'], dayfirst=True)
dtable.head()

,Plantations,AreaType,Monitoring year,Monitoring month,Monitoring day,PLOT,TREE NR,Tree SPECIES,H (m),DBH (cm),REMARKS,Measuring date
44,Tain II,Teak,2019,1,22,1,68.0,Teak,9.50,14.8,None,2019-01-22
45,Tain II,Teak,2019,1,22,1,76.0,Teak,7.50,14.5,None,2019-01-22
46,Tain II,Teak,2019,1,22,1,71.0,Teak,7.25,14.3,None,2019-01-22
47,Tain II,Teak,2019,1,22,1,53.0,Teak,7.50,13.3,None,2019-01-22
48,Tain II,Teak,2019,1,22,1,58.0,Teak,8.00,13.0,None,2019-01-22


### Sort the table by Plot Number, Tree Number and Measuring date

In [28]:
dtable = dtable.sort_values(by=['PLOT', 'TREE NR', 'Measuring date'])
dtable

,Plantations,AreaType,Monitoring year,Monitoring month,Monitoring day,PLOT,TREE NR,Tree SPECIES,H (m),DBH (cm),REMARKS,Measuring date
59,Tain II,Teak,2019,1,22,1,1.0,Teak,7.50,8.9,None,2019-01-22
67638,Tain II,Teak,2020,2,20,1,1.0,Teak,9.75,12.5,None,2020-02-20
111343,Tain II,Teak,2021,4,1,1,1.0,Teak,12.50,16.0,None,2021-04-01
160935,Tain II,Teak,2022,3,8,1,1.0,Teak,13.25,18.3,,2022-03-08
120529,Tain II,Teak,2023,1,4,1,1.0,Teak,14.25,19.0,,2023-01-04
...,...,...,...,...,...,...,...,...,...,...,...,...
160260,Tain II,Teak,2022,2,23,991,79.0,Teak,10.00,11.5,,2022-02-23
130550,Tain II,Teak,2023,2,20,991,79.0,Teak,9.50,13.4,,2023-02-20
186653,Tain II,Teak,2024,2,29,991,79.0,Teak,10.65,14.0,,2024-02-29
100363,Tain II,Teak,2021,3,13,991,80.0,Teak,7.00,8.5,None,2021-03-13


# CALCULATE ANNUAL GROWTH METRICS

In [29]:
dtable['Diameter change'] = dtable.groupby('TREE NR')['DBH (cm)'].diff()
dtable['Height change'] = dtable.groupby('TREE NR')['H (m)'].diff()

In [30]:
dtable

,Plantations,AreaType,Monitoring year,Monitoring month,Monitoring day,PLOT,TREE NR,Tree SPECIES,H (m),DBH (cm),REMARKS,Measuring date,Diameter change,Height change
59,Tain II,Teak,2019,1,22,1,1.0,Teak,7.50,8.9,None,2019-01-22,NaN,NaN
67638,Tain II,Teak,2020,2,20,1,1.0,Teak,9.75,12.5,None,2020-02-20,3.6,2.25
111343,Tain II,Teak,2021,4,1,1,1.0,Teak,12.50,16.0,None,2021-04-01,3.5,2.75
160935,Tain II,Teak,2022,3,8,1,1.0,Teak,13.25,18.3,,2022-03-08,2.3,0.75
120529,Tain II,Teak,2023,1,4,1,1.0,Teak,14.25,19.0,,2023-01-04,0.7,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160260,Tain II,Teak,2022,2,23,991,79.0,Teak,10.00,11.5,,2022-02-23,0.9,2.25
130550,Tain II,Teak,2023,2,20,991,79.0,Teak,9.50,13.4,,2023-02-20,1.9,-0.50
186653,Tain II,Teak,2024,2,29,991,79.0,Teak,10.65,14.0,,2024-02-29,0.6,1.15
100363,Tain II,Teak,2021,3,13,991,80.0,Teak,7.00,8.5,None,2021-03-13,-6.0,-8.50


### Annual Growth

In [31]:
dtable['YearDiff'] = dtable.groupby('TREE NR')['Measuring date'].diff().dt.days / 365.25

# Annual diameter growth
dtable['Annual Diameter Growth'] = dtable['Diameter change'] / dtable['YearDiff']

# Annual height growth
dtable['Annual Height Growth'] = dtable['Height change'] / dtable['YearDiff']

dtable.head()

,Plantations,AreaType,Monitoring year,Monitoring month,Monitoring day,PLOT,TREE NR,Tree SPECIES,H (m),DBH (cm),REMARKS,Measuring date,Diameter change,Height change,YearDiff,Annual Diameter Growth,Annual Height Growth
59,Tain II,Teak,2019,1,22,1,1.0,Teak,7.50,8.9,None,2019-01-22,NaN,NaN,NaN,NaN,NaN
67638,Tain II,Teak,2020,2,20,1,1.0,Teak,9.75,12.5,None,2020-02-20,3.6,2.25,1.078713,3.337310,2.085819
111343,Tain II,Teak,2021,4,1,1,1.0,Teak,12.50,16.0,None,2021-04-01,3.5,2.75,1.111567,3.148707,2.473984
160935,Tain II,Teak,2022,3,8,1,1.0,Teak,13.25,18.3,,2022-03-08,2.3,0.75,0.933607,2.463563,0.803336
120529,Tain II,Teak,2023,1,4,1,1.0,Teak,14.25,19.0,,2023-01-04,0.7,1.00,0.826831,0.846606,1.209437


# COMPUTE BASAL AREA PER TREE

In [32]:
dtable['BA'] = np.pi * (dtable['DBH (cm)'] / 200)**2

# Basal Area Growth
dtable['BAG'] = dtable.groupby('TREE NR')['BA'].diff()

# Annual Basal Area growth
dtable['Annual_BAG'] = dtable['BAG'] / dtable['YearDiff']

dtable.head()

,Plantations,AreaType,Monitoring year,Monitoring month,Monitoring day,PLOT,TREE NR,Tree SPECIES,H (m),DBH (cm),REMARKS,Measuring date,Diameter change,Height change,YearDiff,Annual Diameter Growth,Annual Height Growth,BA,BAG,Annual_BAG
59,Tain II,Teak,2019,1,22,1,1.0,Teak,7.50,8.9,None,2019-01-22,NaN,NaN,NaN,NaN,NaN,0.006221,NaN,NaN
67638,Tain II,Teak,2020,2,20,1,1.0,Teak,9.75,12.5,None,2020-02-20,3.6,2.25,1.078713,3.337310,2.085819,0.012272,0.006051,0.005609
111343,Tain II,Teak,2021,4,1,1,1.0,Teak,12.50,16.0,None,2021-04-01,3.5,2.75,1.111567,3.148707,2.473984,0.020106,0.007834,0.007048
160935,Tain II,Teak,2022,3,8,1,1.0,Teak,13.25,18.3,,2022-03-08,2.3,0.75,0.933607,2.463563,0.803336,0.026302,0.006196,0.006637
120529,Tain II,Teak,2023,1,4,1,1.0,Teak,14.25,19.0,,2023-01-04,0.7,1.00,0.826831,0.846606,1.209437,0.028353,0.002051,0.002480


# STAND-LEVEL ANALYSIS

### Mean DBH, Height and Basal Area

In [33]:
dtable['Measurement Year'] = dtable['Measuring date'].dt.year
plot_growth = dtable.groupby(['PLOT', 'Measurement Year']).agg({
    'Annual Diameter Growth': 'mean',
    'Annual Height Growth': 'mean',
    'Annual_BAG': 'sum',
    'TREE NR': 'nunique'
}).reset_index()

In [34]:
plot_growth

,PLOT,Measurement Year,Annual Diameter Growth,Annual Height Growth,Annual_BAG,TREE NR
0,1,2019,NaN,NaN,0.000000,43
1,1,2020,3.114176,2.231341,0.212824,43
2,1,2021,3.226602,3.549152,0.284604,41
3,1,2022,2.083317,0.890364,0.218715,42
4,1,2023,1.091633,-0.052901,0.127897,42
...,...,...,...,...,...,...
4000,990,2024,0.281355,1.185877,0.106936,42
4001,991,2021,-inf,-inf,-inf,67
4002,991,2022,2.340768,2.919694,0.168620,42
4003,991,2023,2.647366,-0.108105,0.239749,42


In [35]:
plot_growth.to_csv('plot_analysis.csv', encoding='utf-8')